<a href="https://colab.research.google.com/github/NavitB/ATP-Final/blob/master/Body_%2B_Title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports and Installs**

In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import math
from operator import add
import builtins
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.
--2022-01-07 16:21:45--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 13.226.124.24, 13.226.124.87, 13.226.124.26, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|13.226.124.24|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update -qq
!apt install openjdk-8-jdk-headless -qq

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [5]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

Creating RDD from data

In [6]:
pandasDf = pd.read_csv("/content/newdata.csv")
pandasDf = pandasDf.drop(columns = "Unnamed: 0")
df_data = spark.createDataFrame(pandasDf)

id_text = df_data.select('id','text')
id_title = df_data.select('id','title')
id_anchor = df_data.select('id', 'anchor_text')

id_text = id_text.rdd
id_title = id_title.rdd
id_anchor = id_anchor.rdd

inverted_text = InvertedIndex()
inverted_title = InvertedIndex()

In [7]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(id, text):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  tuples = []
  tf_dict = Counter(tokens)
  res = []
  [res.append(x) for x in tokens if x not in res]
  for t in res:
    tuples.append((t,(id,tf_dict[t])))
  return tuples

In [8]:
def count_doc_len(doc_id, text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  return (doc_id,len(tokens))

In [9]:
def tf_for_term_id(doc_id, text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [x for x in tokens if x not in all_stopwords]
  tuples = []
  tf_dict = Counter(tokens)
  res = []
  [res.append(x) for x in tokens if x not in res]
  for t in res:
    tuples.append((t,tf_dict[t]))
  return tuples

In [10]:
word_counts_text = id_text.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_title = id_title.flatMap(lambda x: word_count(x[0], x[1]))

len_docs_text = id_text.map(lambda x: count_doc_len(x[0], x[1]))
len_docs_text = len_docs_text.collectAsMap()
inverted_text.DL = len_docs_text

len_docs_title = id_title.map(lambda x: count_doc_len(x[0], x[1]))
len_docs_title = len_docs_title.collectAsMap()
inverted_title.DL = len_docs_title


In [11]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl)

In [12]:
#create postings and sort by doc_id
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

#anchor?
#filter out rare words - appearing in less than 10 docs
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>10)
postings_filtered_title = postings_title.filter(lambda x: len(x[1])>10)
#anchor?

In [13]:
total_terms_text = postings_filtered_text.flatMapValues(lambda x : x).map(lambda x: (x[0],x[1][1])).reduceByKey(add)
inverted_text.term_total = total_terms_text.collectAsMap()

total_terms_title = postings_filtered_title.flatMapValues(lambda x : x).map(lambda x: (x[0],x[1][1])).reduceByKey(add)
inverted_title.term_total = total_terms_title.collectAsMap()

In [14]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  rdd2 = postings.map(lambda x: (x[0], len(x[1]))) 
  return rdd2

In [15]:
w2df_text = calculate_df(postings_filtered_text)
w2df_title = calculate_df(postings_filtered_title)

w2df_text_dict = w2df_text.collectAsMap()
w2df_title_dict = w2df_title.collectAsMap()

In [16]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings,base_dir):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  rdd2 = postings.map(lambda x: (token2bucket_id(x[0]),x)).groupByKey()
  rdd3 = rdd2.map(lambda x: InvertedIndex.write_a_posting_list(x,base_dir))
  return rdd3


In [17]:
#rdd for postings locations
!mkdir text_pl title_pl 
postings_locs_text = partition_postings_and_write(postings_filtered_text,'text_pl').collect()
postings_locs_title = partition_postings_and_write(postings_filtered_title,'title_pl').collect()


mkdir: cannot create directory ‘text_pl’: File exists
mkdir: cannot create directory ‘title_pl’: File exists


In [18]:
#create super set for all postings locs
super_posting_locs_text = defaultdict(list)
for posting_loc in postings_locs_text:
  for k, v in posting_loc.items():
    super_posting_locs_text[k].extend(v)

super_posting_locs_title = defaultdict(list)
for posting_loc in postings_locs_title:
  for k, v in posting_loc.items():
    super_posting_locs_title[k].extend(v)


#anchor?

In [19]:
inverted_text.posting_locs = super_posting_locs_text
inverted_title.posting_locs = super_posting_locs_title

inverted_text.df = w2df_text_dict
inverted_title.df = w2df_title_dict

inverted_text.write_index('.', 'index_text')
inverted_title.write_index('.', 'index_title')

vector size after tfidf per doc

In [20]:
def calculate_tfidf_doc(tf_term_doc, index):
  DL = index.DL
  N = len(DL)
  df_dict = index.df
  doc_term_tfidf = tf_term_doc.map(lambda x : (x[0] , math.sqrt(builtins.sum([math.pow((tf/DL[x[0]]) * (math.log(N/df_dict[term],10)),2) for term, tf in x[1] if term in df_dict]))))
  return doc_term_tfidf


In [60]:
tf_term_doc_text = id_text.map(lambda x : (x[0], tf_for_term_id(x[0], x[1])))
tfidf_text = calculate_tfidf_doc(tf_term_doc_text, inverted_text)
tfidf_dict_text = tfidf_text.collectAsMap()

# tf_term_doc_title = id_title.map(lambda x : (x[0], tf_for_term_id(x[0], x[1])))
# tfidf_title = calculate_tfidf_doc(tf_term_doc_title, inverted_title)
# tfidf_dict_title = tfidf_title.collectAsMap()


In [61]:
inverted_text.vec_len_doc = tfidf_dict_text

# inverted_title.vec_len_doc = tfidf_dict_title

Reading a posting list?

In [23]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [24]:
def get_posting_gen(index):
    """
    This function returning the generator working with posting list.
    
    Parameters:
    ----------
    index: inverted index    
    """
    words, pls = zip(*index.posting_lists_iter())
    return words,pls

In [41]:
words_text, pls_text = get_posting_gen(inverted_text)
words_title, pls_title = get_posting_gen(inverted_title)

# Search Body

In [26]:
def generate_query_tfidf_vector(query_to_search,index):
    """ 
    Generate a vector representing the query. Each entry within this vector represents a tfidf score.
    The terms representing the query will be the unique terms in the index.

    We will use tfidf on the query as well. 
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the query.    

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.
    
    Returns:
    -----------
    vectorized query with tfidf scores
    """
    
    epsilon = .0000001
    total_vocab_size = len(index.term_total)
    Q = np.zeros((total_vocab_size))
    term_vector = list(index.term_total.keys()) 
    counter = Counter(query_to_search)
    for token in np.unique(query_to_search):
        if token in index.term_total.keys(): #avoid terms that do not appear in the index.               
            tf = counter[token]/len(query_to_search) # term frequency divded by the length of the query
            df = index.df[token]            
            idf = math.log((len(index.DL))/(df+epsilon),10) #smoothing
            try:
                ind = term_vector.index(token)
                Q[ind] = tf*idf                    
            except:
                pass
    return Q
    

In [54]:
 def get_candidate_documents_and_scores(query_to_search,index,words,pls):
    """
    Generate a dictionary representing a pool of candidate documents for a given query. This function will go through every token in query_to_search
    and fetch the corresponding information (e.g., term frequency, document frequency, etc.') needed to calculate TF-IDF from the posting list.
    Then it will populate the dictionary 'candidates.'
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the document.
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: generator for working with posting.
    Returns:
    -----------
    dictionary of candidates. In the following format:
                                                               key: pair (doc_id,term)
                                                               value: tfidf score. 
    """
    candidates = {}
    N = len(index.DL)
    for term in np.unique(query_to_search):  
        if term in words:            
            list_of_doc = pls[words.index(term)]
            normlized_tfidf = [(doc_id,(freq/index.DL[doc_id])*math.log(N/index.df[term],10)) for doc_id, freq in list_of_doc]           
            for doc_id, tfidf in normlized_tfidf:
                candidates[(doc_id,term)] = candidates.get((doc_id,term), 0) + tfidf               
    
    return candidates



In [47]:
def generate_document_tfidf_matrix(query_to_search,index,words,pls):
    """
    Generate a DataFrame `D` of tfidf scores for a given query. 
    Rows will be the documents candidates for a given query
    Columns will be the unique terms in the index.
    The value for a given document and term will be its tfidf score.
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: generator for working with posting.
    Returns:
    -----------
    DataFrame of tfidf scores.
    """
    
    total_vocab_size = len(index.term_total)
    candidates_scores = get_candidate_documents_and_scores(query_to_search,index,words,pls) #We do not need to utilize all document. Only the docuemnts which have corrspoinding terms with the query.
    unique_candidates = np.unique([doc_id for doc_id, freq in candidates_scores.keys()])
    D = np.zeros((len(unique_candidates), total_vocab_size))
    D = pd.DataFrame(D)
    
    D.index = unique_candidates
    D.columns = index.term_total.keys()

    for key in candidates_scores:
        tfidf = candidates_scores[key]
        doc_id, term = key    
        D.loc[doc_id][term] = tfidf


    return D
     

In [29]:
def cosine_similarity(D,Q,index):
    """
    Calculate the cosine similarity for each candidate document in D and a given query (e.g., Q).
    Generate a dictionary of cosine similarity scores 
    key: doc_id
    value: cosine similarity score
    
    Parameters:
    -----------
    D: DataFrame of tfidf scores.

    Q: vectorized query with tfidf scores
    
    Returns:
    -----------
    dictionary of cosine similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: cosine similarty score.
    """
    dict_cos = {}
    for doc_id, row in D.iterrows(): 
      vec_doc = row.to_numpy()
      result = np.dot(vec_doc, Q)/(index.vec_len_doc[doc_id] * np.linalg.norm(Q))
      dict_cos[doc_id] = result
    return dict_cos


In [30]:
def get_top_n(sim_dict,N=3):
    """ 
    Sort and return the highest N documents according to the cosine similarity score.
    Generate a dictionary of cosine similarity scores 
   
    Parameters:
    -----------
    sim_dict: a dictionary of similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: similarity score. We keep up to 5 digits after the decimal point. (e.g., round(score,5))

    N: Integer (how many documents to retrieve). By default N = 3
    
    Returns:
    -----------
    a ranked list of pairs (doc_id, score) in the length of N.
    """
    lst = [(doc_id , builtins.round(score,5)) for doc_id, score in sim_dict.items()]
    return sorted(lst, key = lambda x: x[1], reverse=True)[:N]
    

In [46]:
def get_topN_score_for_queries(queries_to_search,index, words, pls,N=3):
    """ 
    Generate a dictionary that gathers for every query its topN score.
    
    Parameters:
    -----------
    queries_to_search: a dictionary of queries as follows: 
                                                        key: query_id
                                                        value: list of tokens.
    index:           inverted index loaded from the corresponding files.    
    N: Integer. How many documents to retrieve. This argument is passed to the topN function. By default N = 3, for the topN function. 
    
    Returns:
    -----------
    return: a dictionary of queries and topN pairs as follows:
                                                        key: query_id
                                                        value: list of pairs in the following format:(doc_id, score). 
    """
    top_N = {}
    for q_id, q in queries_to_search.items() :
      Q = generate_query_tfidf_vector(q,index)
      D = generate_document_tfidf_matrix(q, index, words, pls)
      top_N[q_id] = get_top_n(cosine_similarity(D, Q, index), N)
    return top_N


# Search Title

In [ ]:
def get_score_for_title(query, index,words, pls):
    
    candidates = {}
    len_query = len(np.unique(query))
    for term in np.unique(query):  
        if term in words:            
            list_of_doc = pls[words.index(term)]
            # normlized_tfidf = [(doc_id,(freq/index.DL[doc_id])*math.log(N/index.df[term],10)) for doc_id, freq in list_of_doc]           
            # for doc_id, tfidf in normlized_tfidf:
            #     candidates[(doc_id,term)] = candidates.get((doc_id,term), 0) + tfidf               
    
    return candidates

  for term in np.unique(query_to_search):


# Testing search

search body

In [57]:
dict_q = {}
# dict_q[1] = ['best', 'marvel', 'movie']
dict_q[1] = ['python']
tfidf_queries_score_train = get_topN_score_for_queries(dict_q, inverted_text,words_text, pls_text, 100)
print(tfidf_queries_score_train)

# tfidf_titles = get_topN_score_for_queries(dict_q, inverted_title,100)
# print(tfidf_titles)


# lst = [57069491, 65967176, 42163310, 878659, 27306717, 41677925, 1074657, 44240443, 17296107, 60952488, 43603241, 22114132, 46208997, 36450985, 41974555, 56289672, 60616450, 59502488, 33038861, 61699239, 61651800, 39368416, 29129051, 55935213, 54537218, 62372638, 60774345, 63090183, 37497391, 51430647, 67229718, 9110929, 61329320, 44254295, 41974496, 200563, 58481694, 48530084, 56289572, 22144990, 612052, 59162931, 55511148, 55511147, 61073786, 59892, 36484005, 36484254, 66423851, 62482816, 1275470, 5676692, 7927053, 60754840, 26999426, 60744481, 56289553, 60463979, 701741, 60283633, 1129847, 36439749, 4451883, 55511155, 22144721, 45359871, 723126, 43655965, 57275457, 12673434, 43867095, 26763420, 39293265, 15003874, 41668588, 61592102, 67063919, 11891433, 64057670, 61513780, 39345917, 67063906, 1221476, 41008758, 60587000, 7729, 2152196, 5027882, 509738, 403585, 26866372, 1339248, 3473503, 4148655]
lst = [23862, 23329, 53672527, 21356332, 4920126, 5250192, 819149, 46448252, 83036, 88595, 18942, 696712, 2032271, 1984246, 5204237, 645111, 18384111, 3673376, 25061839, 271890, 226402, 2380213, 1179348, 15586616, 50278739, 19701, 3596573, 4225907, 19160, 1235986, 6908561, 3594951, 18805500, 5087621, 25049240, 2432299, 381782, 9603954, 390263, 317752, 38007831, 2564605, 13370873, 2403126, 17402165, 23678545, 7837468, 23954341, 11505904, 196698, 34292335, 52042, 2247376, 15858, 11322015, 13062829, 38833779, 7800160, 24193668, 440018, 54351136, 28887886, 19620, 23045823, 43003632, 746577, 1211612, 8305253, 14985517, 30796675, 51800, 964717, 6146589, 13024, 11583987, 57294217, 27471338, 5479462]
lst_docs_returned = [x[0] for x in tfidf_queries_score_train[1]]

lst_equal = [x for x in lst_docs_returned if x in lst]
print(len(lst))
print(len(lst_equal))

# lst_top_6 = [57069491, 65967176, 42163310, 878659, 27306717, 41677925]

lst_indecies = [lst_docs_returned.index(x) for x in lst if x in lst_docs_returned]
print(lst_indecies)


{1: [(23862, 0.86067), (21356332, 0.85709), (18805500, 0.82209), (3673376, 0.80627), (696712, 0.80121), (4920126, 0.6872), (25061839, 0.66749), (1984246, 0.6635), (88595, 0.65193), (46448252, 0.64461), (24193668, 0.64296), (5250192, 0.63494), (18384111, 0.63345), (2032271, 0.63233), (13062829, 0.61265), (13370873, 0.60978), (2564605, 0.60958), (5087621, 0.58553), (5204237, 0.58407), (30796675, 0.56062), (23329, 0.55363), (14985517, 0.54372), (34292335, 0.53946), (11322015, 0.51823), (6146589, 0.51187), (390263, 0.51086), (3596573, 0.5094), (83036, 0.50313), (38007831, 0.48423), (4225907, 0.47154), (9603954, 0.44683), (18942, 0.43328), (7837468, 0.43272), (23045823, 0.42648), (7800160, 0.38195), (271890, 0.37729), (1235986, 0.37336), (57294217, 0.34983), (50278739, 0.34209), (226402, 0.34026), (2380213, 0.3225), (5479462, 0.31138), (645111, 0.30176), (819149, 0.30044), (25049240, 0.28896), (11583987, 0.28795), (53672527, 0.28373), (2247376, 0.28348), (23678545, 0.28238), (28887886, 0.26

search title

In [59]:
dict_q = {}
# dict_q[1] = ['best', 'marvel', 'movie']

dict_q[1] = ['python']


# tfidf_queries_score_train = get_topN_score_for_queries(dict_q, inverted_text,100)
# print(tfidf_queries_score_train)

tfidf_titles = get_topN_score_for_queries(dict_q, inverted_title, words_title, pls_title ,100)
print(tfidf_titles)


# lst = [57069491, 65967176, 42163310, 878659, 27306717, 41677925, 1074657, 44240443, 17296107, 60952488, 43603241, 22114132, 46208997, 36450985, 41974555, 56289672, 60616450, 59502488, 33038861, 61699239, 61651800, 39368416, 29129051, 55935213, 54537218, 62372638, 60774345, 63090183, 37497391, 51430647, 67229718, 9110929, 61329320, 44254295, 41974496, 200563, 58481694, 48530084, 56289572, 22144990, 612052, 59162931, 55511148, 55511147, 61073786, 59892, 36484005, 36484254, 66423851, 62482816, 1275470, 5676692, 7927053, 60754840, 26999426, 60744481, 56289553, 60463979, 701741, 60283633, 1129847, 36439749, 4451883, 55511155, 22144721, 45359871, 723126, 43655965, 57275457, 12673434, 43867095, 26763420, 39293265, 15003874, 41668588, 61592102, 67063919, 11891433, 64057670, 61513780, 39345917, 67063906, 1221476, 41008758, 60587000, 7729, 2152196, 5027882, 509738, 403585, 26866372, 1339248, 3473503, 4148655]
lst = [23862, 23329, 53672527, 21356332, 4920126, 5250192, 819149, 46448252, 83036, 88595, 18942, 696712, 2032271, 1984246, 5204237, 645111, 18384111, 3673376, 25061839, 271890, 226402, 2380213, 1179348, 15586616, 50278739, 19701, 3596573, 4225907, 19160, 1235986, 6908561, 3594951, 18805500, 5087621, 25049240, 2432299, 381782, 9603954, 390263, 317752, 38007831, 2564605, 13370873, 2403126, 17402165, 23678545, 7837468, 23954341, 11505904, 196698, 34292335, 52042, 2247376, 15858, 11322015, 13062829, 38833779, 7800160, 24193668, 440018, 54351136, 28887886, 19620, 23045823, 43003632, 746577, 1211612, 8305253, 14985517, 30796675, 51800, 964717, 6146589, 13024, 11583987, 57294217, 27471338, 5479462]

lst_docs_returned = [x[0] for x in tfidf_titles[1]]

lst_equal = [x for x in lst_docs_returned if x in lst]
print(len(lst))
print(len(lst_equal))

# lst_top_6 = [57069491, 65967176, 42163310, 878659, 27306717, 41677925]

lst_indecies = [lst_docs_returned.index(x) for x in lst if x in lst_docs_returned]
print(lst_indecies)


{1: [(18942, 1.0), (19160, 1.0), (19701, 1.0), (23862, 1.0), (83036, 1.0), (88595, 1.0), (196698, 1.0), (271890, 1.0), (317752, 1.0), (696712, 1.0), (819149, 1.0), (2032271, 1.0), (2380213, 1.0), (2403126, 1.0), (2432299, 1.0), (3594951, 1.0), (3596573, 1.0), (4920126, 1.0), (5087621, 1.0), (5204237, 1.0), (5250192, 1.0), (6146589, 1.0), (6908561, 1.0), (7800160, 1.0), (7837468, 1.0), (9603954, 1.0), (11505904, 1.0), (11583987, 1.0), (13062829, 1.0), (13370873, 1.0), (14985517, 1.0), (15586616, 1.0), (18805500, 1.0), (21356332, 1.0), (23678545, 1.0), (24193668, 1.0), (25049240, 1.0), (25061839, 1.0), (30796675, 1.0), (43003632, 1.0), (46448252, 1.0), (53672527, 1.0), (54351136, 1.0), (3673376, 0.7891), (645111, 0.75788), (746577, 0.68566)]}
78
46
[3, 41, 33, 17, 20, 10, 40, 4, 5, 0, 9, 11, 19, 44, 43, 37, 7, 12, 31, 2, 16, 1, 22, 15, 32, 18, 36, 14, 25, 8, 29, 13, 34, 24, 26, 6, 28, 23, 35, 42, 39, 45, 30, 38, 21, 27]
